In [ ]:
from __future__ import print_function
import os
import numpy as np
from tqdm import tqdm
# import pandas as pd
import matplotlib.pyplot as plt

__autor__ = 'Kyosuke Yamamoto (kyon)'
__date__ = '31 Dec 2017'

# Sumarize results

In [ ]:
import re
import glob

results = {}

for dir in glob.glob('output/*'):

    # skipped folders
    if os.path.splitext(dir)[-1] == '.py':
        continue
    elif os.path.basename(dir) == 'old':
        continue

    # log exists?
    logpath = os.path.join(dir, 'log')
    if not os.path.exists(logpath):
        continue
        
    #init result
    key = os.path.basename(dir)
    results[key] = {}

    # load log
    with open(logpath, 'r') as f:
        log = f.readlines()
    for line in log:
        if 'Namespace' in line:
            # print(line)
            results[key]['batchsize'] = int(re.search('batchsize=([0-9]{,3}),', line).group(1))
            results[key]['debug'] = str(re.search('debug=(.{,5}),', line).group(1))
            results[key]['epochs'] = int(re.search('epochs=([0-9]{,4}),', line).group(1))
            results[key]['threshold'] = int(re.search('threshold=([0-9]{,3})', line).group(1))
            try:
                results[key]['optimizer'] = str(re.search("optimizer='(.{,10})',", line).group(1))
                results[key]['input'] = str(re.search("input=\[(.+)\],", line).group(1))
                results[key]['model'] = str(re.search("model='(.{,3})',", line).group(1))
            except:
                pass
        elif 'acc' in line:
            match = re.search('\>\s.+\:\s(.*)', line)
            mae = float(match.group(1))
            results[key]['MAE'] = round(mae, 4)
            
results = pd.DataFrame(results).T
results.index = pd.to_datetime(results.index, format='%m-%d-%H-%M-%S')
results

In [ ]:
#easydict
import easydict
args = easydict.EasyDict({
        'path': './output/03-24-21-21-23',
})

#init
outdir = args.path

#accuracies
with open(os.path.join(args.path, 'log')) as f:
    lines = f.readlines()
    for line in lines:
        if line.find('Test') >= 0:
            print(line[:-1])

#ml.png
from PIL import Image
image = Image.open(os.path.join(args.path, 'ml.png'))
image.resize((500, 500))

In [ ]:
from vis.utils import utils
from keras import activations
from keras.models import load_model
from vis.visualization import visualize_activation
from vis.input_modifiers import Jitter

model = load_model(os.path.join(args.path, 'model.h5'))
model.summary()

# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot
# SVG(model_to_dot(model).create(prog='dot', format='svg'))

# Layer & filter visualization

In [ ]:
#
# Denase Layer Visualization
#

result = {}

layer_idx = utils.find_layer_idx(model, 'dense_2')   # last layer
result['normal'] = visualize_activation(model, layer_idx, filter_indices=0)
result['iter_50'] = visualize_activation(model, layer_idx, filter_indices=0, max_iter=50, verbose=False)
result['iter_500'] = visualize_activation(model, layer_idx, filter_indices=0, max_iter=500, verbose=False)
# result['iter_1000'] = visualize_activation(model, layer_idx, filter_indices=0, max_iter=1000, verbose=False)
result['Jitter'] = visualize_activation(model, layer_idx, filter_indices=0, max_iter=50, input_modifiers=[Jitter(16)])
#TODO(kyon): plot history

fig, axes = plt.subplots(1, len(result.keys()))
for i, (title, img) in enumerate(result.items()):
    ax = axes[i]
    # ax.axis('off')
    ax.set_title(title)
    cax = ax.imshow(img[..., 0], interpolation='nearest', aspect='auto',
                   clim=(np.array(list(result.values())).min(), np.array(list(result.values())).max()))
fig.colorbar(cax)
fig.tight_layout()
fig.savefig(os.path.join(outdir, 'dense_layer_vis.pdf'))

In [ ]:
#
# Conv filter visualization
#

# TODO: Try Jitter (already tried but could not understand the result...)

from vis.visualization import get_num_filters

#Visualize all filters in this layer
layer_idx = utils.find_layer_idx(model, 'conv2d_1')
filters = np.arange(get_num_filters(model.layers[layer_idx]))

#generate input image for each filter
vis_images = []
for idx in tqdm(filters):
    img = visualize_activation(model, layer_idx, filter_indices=idx)
    vis_images.append(img)

#plot
fig, axes = plt.subplots(3, 12)
for i, (img, ax) in enumerate(zip(vis_images, axes.flatten())):
    ax.axis('off')
    ax.set_title(f'l{i}')
    cax = ax.imshow(img[..., 0], interpolation='nearest',
                    aspect='auto', clim=(np.array(vis_images).min(), np.array(vis_images).max()))
fig.colorbar(cax)
# fig.tight_layout()
fig.savefig(os.path.join(outdir, 'conv_layer_vis.pdf'))

# Attension Map

In [ ]:
import matplotlib.cm as cm
from vis.visualization import visualize_saliency

x_train = np.load(os.path.join(args.path, 'x_train.npy'))
y_train = np.load(os.path.join(args.path, 'y_train.npy'))
r_train = np.load(os.path.join(args.path, 'r_train.npy'))

## Saliency

In [ ]:
#
# saliency of y=0~1
#

fig, axes = plt.subplots(10, 5, figsize=(10, 20))

for cnt, y_th in enumerate(tqdm(np.arange(0, 1, .1))):

    x = x_train[(y_train.flatten() >= y_th-.03) & (y_train.flatten() < y_th+.03)][0]
    r = r_train[(y_train.flatten() >= y_th-.03) & (y_train.flatten() < y_th+.03)][0]
    y = y_train[(y_train.flatten() >= y_th-.03) & (y_train.flatten() < y_th+.03)][0][0]
    
    #plot inputs
    ax = axes[cnt, 0]
    cax = ax.imshow(x[..., 0], cmap='jet', clim=(0, 1), interpolation='nearest', aspect='auto')
    ax.axis('off')
    ax.set_title(f'input (y={y:.2f})')
    fig.colorbar(cax, orientation='horizontal', ax=ax)
    
    #plot DVI
    ax = axes[cnt, 4]
    dvi = r[:, 0]    # fixed!
    ax.plot(dvi, range(len(dvi)))
    ax.set_ylim(350, 0)
    # cax = ax.imshow(dvi, cmap='jet', clim=(0, 2), interpolation='nearest', aspect='auto')
    # ax.axis('off')
    # ax.set_title('DVI')
    # fig.colorbar(cax, orientation='horizontal', ax=ax, ticks=[0, 1, 2])
    
    #plot saliencies
    modifiers = [None, 'negate', 'small_values']
    for j, modifier in enumerate(modifiers):

        layer_idx = utils.find_layer_idx(model, 'dense_2')
        grads = visualize_saliency(model, layer_idx, filter_indices=0,
                                   seed_input=x, grad_modifier=modifier)
        x = axes[cnt, j+1]
        ax.axis('off')
        cax = ax.imshow(grads, cmap='jet', clim=(0, 1), interpolation='nearest', aspect='auto')
        fig.colorbar(cax, orientation='horizontal', ax=ax)
        
        if cnt == 0:
            ax.set_title('positive' if modifier is None else modifier)

fig.tight_layout()
fig.savefig(os.path.join(outdir, 'saliency_vis_y0-1.png'), dpi=200)

In [ ]:
#
# saliency of fixed-y
#

fig, axes = plt.subplots(10, 5, figsize=(10, 20))

for cnt in tqdm(range(10)):

    y_th = 0.6
    x = x_train[(y_train.flatten() >= y_th-.03) & (y_train.flatten() < y_th+.03)][cnt]
    r = r_train[(y_train.flatten() >= y_th-.03) & (y_train.flatten() < y_th+.03)][cnt]
    y = y_train[(y_train.flatten() >= y_th-.03) & (y_train.flatten() < y_th+.03)][cnt][0]
    
    #plot inputs
    ax = axes[cnt, 0]
    cax = ax.imshow(x[..., 0], cmap='jet', clim=(0, 1), interpolation='nearest', aspect='auto')
    ax.axis('off')
    ax.set_title(f'input (y={y:.2f})')
    fig.colorbar(cax, orientation='horizontal', ax=ax)
    
    #plot DVI
    ax = axes[cnt, 4]
    dvi = r[:, 0]    # fixed!
    ax.plot(dvi, range(len(dvi)))
    ax.set_ylim(350, 0)
    
    #plot saliencies
    modifiers = [None, 'negate', 'small_values']
    for j, modifier in enumerate(modifiers):

        layer_idx = utils.find_layer_idx(model, 'dense_2')
        grads = visualize_saliency(model, layer_idx, filter_indices=0,
                                   seed_input=x, grad_modifier=modifier)
        ax = axes[cnt, j+1]
        ax.axis('off')
        cax = ax.imshow(grads, cmap='jet', clim=(0, 1), interpolation='nearest', aspect='auto')
        fig.colorbar(cax, orientation='horizontal', ax=ax)
        
        if cnt == 0:
            ax.set_title('positive' if modifier is None else modifier)
    
fig.tight_layout()
fig.savefig(os.path.join(outdir, 'saliency_vis_fixed_y.png'), dpi=200)

In [ ]:
#
# saliency of all images
# relation between saliency and dvi
#

from collections import defaultdict

modifiers = {'positive': None, 'negate': 'negate', 'small_values': 'small_values'}
layer_idx = utils.find_layer_idx(model, 'dense_2')
saliencies = {}

for modifier_tile, modifier in modifiers.items():
    saliency = []
        
    for cnt in tqdm(range(1000)):
        
        try:
            i = np.random.randint(0, len(x_train))

            x = x_train[i]
            r = r_train[i]
            y = y_train[i][0]
            dvi = r[:, 0, 0]    # fixed! (DVI is in 0th column)

        except:
            print('failed in hoge')
            pass
        
        try:
            grads = visualize_saliency(model, layer_idx, filter_indices=0, seed_input=x, grad_modifier=modifier)
        except:
            print('failed in vis')
            pass

        try:
            for col in range(x.shape[1]):
                saliency.append(pd.DataFrame(
                    {'column': col, 'image_num': i, 'dvi': dvi, 'saliency': grads[:, col]}))
        except:
            print('failed in dict')
            pass
            
        #debug
        # if i > 3:
        #     break
            
    #save as dict
    saliencies[modifier_tile] = saliency

In [ ]:
for modifier_tile, saliency in saliencies.items():
    
    print(modifier_tile)
    df = pd.concat(saliency)
    fig, axes = plt.subplots(2, 3)

    for i, column in enumerate(df.column.unique()):
        ax = axes.flatten()[i]
        df[df.column == column].plot.scatter(x='dvi', y='saliency', ax=ax)
        ax.set_title(f'column={column}')

    # fig.subtitle(modifier_tile)
    fig.tight_layout()

## CAM

In [ ]:
#cam

from vis.visualization import visualize_cam

fig, axes = plt.subplots(10, 4, figsize=(10, 20))
cnt = 0

for y in tqdm(np.arange(0, 1, .1)):

    x = x_train[(y_train.flatten() >= y-.03) & (y_train.flatten() < y+.03)][0]
    y = y_train[(y_train.flatten() >= y-.03) & (y_train.flatten() < y+.03)][0][0]
    ax = axes[cnt, 0]
    cax = ax.imshow(x[..., 0], cmap='jet', clim=(0, 1), interpolation='nearest', aspect='auto')
    ax.axis('off')
    ax.set_title(f'input (y={y:.2f})')
    fig.colorbar(cax, orientation='horizontal', ax=ax)
    
    modifiers = [None, 'negate', 'small_values']

    for j, modifier in enumerate(modifiers):

        layer_idx = utils.find_layer_idx(model, 'dense_2')
        grads = visualize_cam(model, layer_idx, filter_indices=0,
                              seed_input=x, grad_modifier=modifier)
        #TODO: grads is 3-channels image.
        ax = axes[cnt, j+1]
        ax.axis('off')
        cax = ax.imshow(grads, cmap='jet', clim=(0, 255), interpolation='nearest', aspect='auto')
        fig.colorbar(cax, orientation='horizontal', ax=ax)
        
        if cnt == 0:
            ax.set_title('positive' if modifier is None else modifier)
    
    cnt += 1

fig.tight_layout()
fig.savefig(os.path.join(outdir, 'cam_vis.pdf'))